<a href="https://colab.research.google.com/github/MajedeR/mechanical-stress-prediction/blob/main/stress_prediction_6061.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as np
import numpy as np